In [2]:
import pandas as pd
df = pd.read_csv("filtered.tsv", sep="\t", index_col='Unnamed: 0')
df = df.drop(columns=['similarity', 'lenght_diff'])

In [3]:
cond = df["trn_tox"] > df["ref_tox"]
df.loc[cond, ["reference", "translation"]] = df.loc[cond, ["translation", "reference"]].values
df.loc[cond, ["ref_tox", "trn_tox"]] = df.loc[cond, ["trn_tox", "ref_tox"]].values

In [5]:
df = df[((df["ref_tox"] > 0.8) & (df["trn_tox"] < 0.1))]

In [6]:
df = df[((df['reference'].str.len() > 30) & (df['translation'].str.len() > 20))]

In [11]:
sorted_df = df.sort_values(by=['ref_tox', 'trn_tox'],
                           key=lambda x: abs(df['ref_tox'] - df['trn_tox']))

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sorted_df, test_size=0.2, random_state=42, shuffle=False)

In [18]:
test

,reference,translation,ref_tox,trn_tox
237156,You shut up! This is what the Alphabet crimes ...,this is what these alphabet crimes are... guns!,0.997347,0.001350
427656,mixed with her tears and milk of her breasts.,mixed with a mother's tears and baked in her b...,0.996332,0.000334
293162,"a good girl will be good everywhere, and a foo...","A nice girl is a nice girl anywhere, and a bum...",0.999387,0.003389
26186,"I know, Nick Fisher spent eight years in jail ...",I know because Nick spent eight years in Dade ...,0.996274,0.000277
385770,"Shit, he been stealing it from us anyway.",he's stealing it from us all the time.,0.996249,0.000251
...,...,...,...,...
481493,why did we come to this stupid country?,Why did we ever come to this country?,0.999719,0.000043
429942,"How do you like your stupid game now, Tom?","how do you like your game now, Tom?",0.999720,0.000043
506123,"those idiots from the station, I'm sure it's t...","These are the types of station, there are those.",0.999716,0.000039
155243,we started our own firm that day and said good...,That's the day we started our own business and...,0.999718,0.000039


In [34]:
for_prompt_high = train[-5000:].sample(5)

med = len(sorted_df) // 2
for_prompt_medium = train[(med-2500):(med+2500)].sample(1)

for_prompt_low = train[:5000].sample(1)

In [53]:
for_prompt = pd.concat([for_prompt_high, for_prompt_medium, for_prompt_low])

In [54]:
for_test = test[:1000].sample(1).iloc[0]

In [57]:
prompt = '[INST] Make text NON TOXIC according to the examples. Write ONLY Non-toxic text as an output! It could not be no response. [/INST] \n'
for i, e in for_prompt.iterrows():
    prompt += f"Toxic text: {e['reference']}\nNon-toxic text: {e['translation']}\n\n"
prompt += f"Toxic text: {for_test['reference']}\nNon-toxic text: "

In [58]:
print(prompt)

<s> [INST] Make text NON TOXIC according to the examples. Write ONLY Non-toxic text as an output! It could not be no response. [/INST] 
Toxic text: Bourne will be useless to me as a dead man.
Non-toxic text: Bourne dead gives me nothing.

Toxic text: You know, I understand the whole entourage thing, but I don't get why a nice guy like Scott has to hang out with those two parasites.
Non-toxic text: you know, I get the whole thing with the escort, but I don't understand why a nice guy like Scott is throwing two parasions with them.

Toxic text: I think he'd go home not feeling like a piece of crap.
Non-toxic text: I think he would go home and he didn't feel like an onuce.

Toxic text: She created you to deal with the shrinks and the suits... and all their bureaucratic bullshit.
Non-toxic text: she created you to deal with psychiatrists and other shots and all their bureaucratic miscarriages.

Toxic text: Just because you break into people's homes... doesn't mean you have to look like a f

In [52]:
for_test['translation']

"He's an Irishman who doesn't drink, doesn't smoke and doesn't chase dames."

In [80]:
sorted_df[:5000]

,reference,translation,ref_tox,trn_tox
363250,You were so mean.,you were so pompous.,0.803552,0.198702
533309,She hates my mom.,he hates my mom.,0.804029,0.199163
529313,His soldiers and that devil-spawned Spaniard t...,his soldiers and the devil-obsessed Spanish ty...,0.801874,0.195912
335023,one side brewed and the other completely black.,"Boiled one side, and black the next.",0.805712,0.199540
180269,Intellectually suited to carry weapons and die...,intellectually destined to carry guns and die ...,0.804034,0.196836
...,...,...,...,...
358685,what the hell is so important that you have to...,"Lex, what is so damn important you had to pull...",0.847703,0.087800
382554,the hell didn't.,The hell I didn't.,0.814432,0.054525
304361,"Not salty, doesn't smell good, not sweet, tast...","it's not salty, it smells funny, it's not swee...",0.913233,0.153304
539388,You will curse us all.,you will bring us all the curse.,0.808365,0.048431


In [47]:
part[["reference", "translation"]] = part[["translation", "reference"]]
part[["ref_tox", "trn_tox"]] = part[["trn_tox", "ref_tox"]]

<ipython-input-47-cbd9b4aa8024>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part[["reference", "translation"]] = part[["translation", "reference"]]
<ipython-input-47-cbd9b4aa8024>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part[["ref_tox", "trn_tox"]] = part[["trn_tox", "ref_tox"]]


In [48]:
part

,reference,translation,ref_tox,trn_tox
0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.981983,0.014195
1,you're becoming disgusting.,Now you're getting nasty.,0.999039,0.065473
3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.994215,0.053362
4,I have orders to kill her.,I've got orders to put her down.,0.999348,0.009402
8,"Briggs, what the hell is going on?","Briggs, what the hell's happening?",0.841071,0.159096
...,...,...,...,...
577763,"oh, and hell.","Oh, crud.",0.807995,0.019170
577764,"execute him, Chief.","Carry on, Chief.",0.998043,0.000092
577766,tell the doctor I'm going to throw the poo.,Tell the doctor I'm dropping a deuce.,0.874277,0.000456
577772,you didn't know that Estelle stole your fish f...,You didn't know that Estelle had stolen some f...,0.949143,0.000121


In [31]:
def move(obj):
    return obj

In [4]:
references = list(df['reference'])
translations = list(df['translation'])
A = references + translations

ref_tox = list(df['ref_tox'])
trn_tox = list(df['trn_tox'])
A_tox = ref_tox + trn_tox

In [7]:
df

,reference,translation,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.009402,0.999348
...,...,...,...,...
577772,You didn't know that Estelle had stolen some f...,you didn't know that Estelle stole your fish f...,0.000121,0.949143
577773,It'il suck the life out of you!,you'd be sucked out of your life!,0.996124,0.215794
577774,"I can't fuckin' take that, bruv.",I really can't take this.,0.984538,0.000049
577775,They called me a fucking hero. The truth is I ...,"they said I was a hero, but I didn't care.",0.991945,0.000124


In [9]:
df.loc[577775].reference

"They called me a fucking hero. The truth is I didn't care anymore."

In [124]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# load tokenizer and model weights
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

# prepare the input


# inference


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [125]:
import random
sample = random.sample(range(len(A)), k=64)
sampled_A = [A[e] for e in sample]
sampled_A_tox = [A_tox[e] for e in sample]

In [12]:
from torch.nn import functional as F

In [126]:
sampled_A

['You want me to take him to the hole?',
 'he wants to take my sword from Sa Yin!',
 "He was gonna kill me, wasn't he?",
 "You're not going to kill me.",
 "There's only one person Bold enough to steal from the wicked witch.",
 'he killed my wife and he took my memory.',
 'The French have a president, Sarkozy, whose wife fucked Mick Jagger.',
 'And I, blind, vile... and wretched... accepted it all.',
 'he could stand behind you, and you would find out after he died!',
 "I have to convince her parents I'm not a complete klutz.",
 "But the saddest thing is that already as a child, I remember it exactly, already as an eight year old I desperately longed for a woman's crotch and I still do.",
 'For official monkeys like you.',
 "We're financing the Nazi war effort!",
 '"Give it up, I\'m a bad hot witch" is okay.',
 'This may include an examination of your breasts and/or internal examination, if necessary.',
 'that was crazy, Henri.',
 'That child of Hell had nothing human; nothing lived in 

In [127]:
batch = tokenizer(sampled_A, return_tensors='pt', padding=True)
sm = F.softmax(model(**batch).logits, dim=1)

In [128]:
sm[:, 1]

tensor([1.1950e-04, 1.3154e-03, 9.9244e-01, 4.5106e-03, 9.9489e-01, 6.4349e-01,
        9.9534e-01, 8.9696e-01, 5.6884e-03, 1.1494e-01, 9.9112e-01, 9.6644e-01,
        3.0749e-04, 9.0413e-01, 2.9637e-01, 5.8666e-04, 9.7305e-01, 6.3886e-02,
        9.9312e-01, 9.8282e-01, 9.8058e-04, 9.7547e-01, 9.9385e-01, 9.9697e-01,
        9.7384e-01, 9.9857e-01, 9.9455e-01, 1.3359e-02, 9.9850e-01, 6.3774e-03,
        4.6834e-05, 1.4676e-01, 9.8864e-01, 2.3837e-02, 9.9770e-01, 3.3312e-03,
        9.3614e-02, 9.9795e-01, 3.8579e-01, 2.3048e-03, 1.4461e-03, 9.9444e-01,
        9.9486e-01, 9.8974e-01, 5.9051e-04, 9.9814e-01, 2.6410e-03, 9.8740e-01,
        9.9081e-01, 5.3516e-03, 9.9518e-01, 4.7640e-04, 9.8507e-01, 3.9103e-01,
        9.4024e-01, 9.3495e-01, 9.9871e-01, 7.5652e-01, 1.7666e-02, 1.5049e-04,
        6.3023e-02, 9.0903e-01, 1.2305e-01, 9.9457e-01],
       grad_fn=<SelectBackward0>)

In [117]:
import numpy as np
import torch

In [129]:


F.mse_loss(sm[:, 1], torch.tensor(sampled_A_tox))

tensor(0.0282, grad_fn=<MseLossBackward0>)